In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from os import listdir
from os.path import isfile, join
import glob 
import os
import re
import matplotlib.pyplot as plt
import little_mallet_wrapper as lmw
from pathlib import Path
from collections import Counter 
from wordcloud import WordCloud
import nltk
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models import CoherenceModel
import gensim.models

In [2]:
df = pd.read_csv('CR_dataset.csv')
df.head()

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Year       Company                                report_preprocessed  \
0  2019       NYSE_IT  corporate responsibility report shaping succes...   
1  2020       NYSE_IT  inc and/or affiliates rights reserved impact c...   
2  2021       NYSE_IT  focused flexible interconnected corporate resp...   
3  2021  NASDAQ_CMCSA  nbcuniversal impact report impact report nbcun...   
4  2020  NASDAQ_CMCSA  nbcuniversal values reporttogether build inter...   

  company_name  nb_of_reports_in_dataset Global 500 Global 100 FAANG  \
0      Gartner                         3         no         no    no   
1      Gartner                         3         no         no    no   
2      Gartner                         3         no         no    no   
3      Comcast                         9        yes        yes    no   
4      Comcast                         9        yes        yes    no   

   unique_words  total_word_count  \
0          3892             18341   
1          3177             11049   
2          5424             22048   
3          5753             33463   
4          7026             73374   

                                    report_tokenized  \
0  ['corporate', 'responsibility', 'report', 'sha...   
1  ['inc', 'and/or', 'affiliates', 'rights', 'res...   
2  ['focused', 'flexible', 'interconnected', 'cor...   
3  ['nbcuniversal', 'impact', 'report', 'impact',...   
4  ['nbcuniversal', 'values', 'reporttogether', '...   

                                   report_lemmatized  \
0  corporate responsibility report shape successf...   
1  inc and/or affiliate right reserve impact conn...   
2  focus flexible interconnected corporate respon...   
3  nbcuniversal impact report impact report nbcun...   
4  nbcuniversal value reporttogether build inters...   

                                 report_lemmatized_2  \
0  ['corporate', 'responsibility', 'report', 'sha...   
1  ['inc', 'and/or', 'affiliate', 'right', 'reser...   
2  ['focus', 'flexible', 'interconnected', 'corpo...   
3  ['nbcuniversal', 'impact', 'report', 'impact',...   
4  ['nbcuniversal', 'value', 'reporttogether', 'b...   

                          report_lemmatized_filtered  
0  ['corporate', 'responsibility', 'report', 'sha...  
1  ['inc', 'and/or', 'affiliate', 'right', 'reser...  
2  ['focus', 'flexible', 'corporate', 'responsibi...  
3  ['nbcuniversal', 'impact', 'report', 'impact',...  
4  ['nbcuniversal', 'value', 'build', 'intersecti...

In [3]:
df['report_lemmatized_filtered2'] = df['report_lemmatized_filtered'].map(lambda x: " ".join(eval(x)))

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df =20, max_features=7300, use_idf=True)
tfidf = tfidf_vectorizer.fit_transform(df['report_lemmatized_filtered2'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
doc_term_matrix_tfidf = pd.DataFrame(tfidf.toarray(), columns=list(tfidf_feature_names))
doc_term_matrix_tfidf

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


abatement  abbreviation  abc     abide   ability      able  abled  \
0           0.0           0.0  0.0  0.007515  0.015961  0.016064    0.0   
1           0.0           0.0  0.0  0.003547  0.005651  0.009479    0.0   
2           0.0           0.0  0.0  0.003126  0.009959  0.008353    0.0   
3           0.0           0.0  0.0  0.000000  0.007623  0.011509    0.0   
4           0.0           0.0  0.0  0.000000  0.017575  0.013267    0.0   
...         ...           ...  ...       ...       ...       ...    ...   
1091        0.0           0.0  0.0  0.000000  0.004854  0.000000    0.0   
1092        0.0           0.0  0.0  0.000000  0.008788  0.000000    0.0   
1093        0.0           0.0  0.0  0.007464  0.000000  0.011967    0.0   
1094        0.0           0.0  0.0  0.000000  0.011528  0.000000    0.0   
1095        0.0           0.0  0.0  0.000000  0.002123  0.000000    0.0   

      abnormal  abolition  aboriginal  ...   yr  yrs  ystem  zation   zealand  \
0          0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
1          0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.004384   
2          0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
3          0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
4          0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
...        ...        ...         ...  ...  ...  ...    ...     ...       ...   
1091       0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
1092       0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
1093       0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
1094       0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.000000   
1095       0.0        0.0         0.0  ...  0.0  0.0    0.0     0.0  0.004940   

          zero  zhejiang  zhuhai  zinc      zone  
0     0.002438       0.0     0.0   0.0  0.003687  
1     0.002301       0.0     0.0   0.0  0.000000  
2     0.008112       0.0     0.0   0.0  0.000000  
3     0.051225       0.0     0.0   0.0  0.098611  
4     0.026841       0.0     0.0   0.0  0.000000  
...        ...       ...     ...   ...       ...  
1091  0.035585       0.0     0.0   0.0  0.000000  
1092  0.042950       0.0     0.0   0.0  0.000000  
1093  0.009685       0.0     0.0   0.0  0.000000  
1094  0.000000       0.0     0.0   0.0  0.000000  
1095  0.000000       0.0     0.0   0.0  0.000000  

[1096 rows x 6553 columns]

In [5]:
no_top_words = 10

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                          for i in topic.argsort()[:-no_top_words - 1:-1]]))

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
nmf = NMF(n_components=29, random_state=0, alpha=.1, init='nndsvd').fit(tfidf)
display_topics(nmf, tfidf_feature_names, no_top_words)

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Applications/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


Topic 0:
supplier chain supply risk audit material safety mineral rba assessment
Topic 1:
printer inc substance recycle cartridge production inkjet site material chemical
Topic 2:
diversity student woman learn organization digital privacy education inclusion school
Topic 3:
programme plc organisation recognise centre digital emission target staff key
Topic 4:
sdi krw battery pdp overseas rechargeable material cheonan issue sustainability
Topic 5:
water ton metric emission scope gallon gas fab facility consumption
Topic 6:
renewable emission carbon center scope electricity power ghg climate project
Topic 7:
csr ict human society governance supplier promote issue stakeholder network
Topic 8:
cent member wireless million target tax site footnote wireline labour
Topic 9:
solution market branch sale project sustainable installers offer growth logistics
Topic 10:
sustainability risk sustainable emission compliance stakeholder optical network future ict
Topic 11:
bang csr sound corruption lab

- topics seem more distinct compared to LDA
- it also runs much faster
- however, still a lot of noise: I should remove currencies (euro, yuan, krw, ...)

In [7]:
# adding the topics to the df
nmf_topic_values = nmf.transform(tfidf)
df['nmf_topics'] = nmf_topic_values.argmax(axis=1)

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Applications/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


In [26]:
# Looking at the top documents for a specific topic
df.loc[df['nmf_topics']==26]

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Year      Company                                report_preprocessed  \
321  2009  NASDAQ_FLEX  beyond responsibility building community corpo...   
498  2014     NYSE_SNE  oefy cpvu fqpsujoh bobhfnfou fttbhf poz shboj ...   
500  2015     NYSE_SNE  csr report fqpsujoh oefy cpvu fqpsujoh bobhfnf...   
502  2017     NYSE_SNE  csr report index csr reporting management mess...   
505  2018     NYSE_SNE  sustainability report nindex cpvu uif vtubjobc...   
857  2011       huawei  investment holding ltd corporate sustainabilit...   

         company_name  nb_of_reports_in_dataset Global 500 Global 100 FAANG  \
321              Flex                        10         no         no    no   
498  Sony Corporation                        22        yes         no    no   
500  Sony Corporation                        22        yes         no    no   
502  Sony Corporation                        22        yes         no    no   
505  Sony Corporation                        22        yes         no    no   
857            huawei                        13        yes        yes    no   

     unique_words  total_word_count  \
321          2359              7159   
498          2194              9664   
500          4204             16247   
502          5826             29401   
505          4495             17741   
857          1743              4550   

                                      report_tokenized  \
321  ['beyond', 'responsibility', 'building', 'comm...   
498  ['oefy', 'cpvu', 'fqpsujoh', 'bobhfnfou', 'ftt...   
500  ['csr', 'report', 'fqpsujoh', 'oefy', 'cpvu', ...   
502  ['csr', 'report', 'index', 'csr', 'reporting',...   
505  ['sustainability', 'report', 'nindex', 'cpvu',...   
857  ['investment', 'holding', 'ltd', 'corporate', ...   

                                     report_lemmatized  \
321  beyond responsibility building community corpo...   
498  oefy cpvu fqpsujoh bobhfnfou fttbhf poz shboj ...   
500  csr report fqpsujoh oefy cpvu fqpsujoh bobhfnf...   
502  csr report index csr report management message...   
505  sustainability report nindex cpvu uif vtubjobc...   
857  investment hold ltd corporate sustainability r...   

                                   report_lemmatized_2  \
321  ['beyond', 'responsibility', 'building', 'comm...   
498  ['oefy', 'cpvu', 'fqpsujoh', 'bobhfnfou', 'ftt...   
500  ['csr', 'report', 'fqpsujoh', 'oefy', 'cpvu', ...   
502  ['csr', 'report', 'index', 'csr', 'report', 'm...   
505  ['sustainability', 'report', 'nindex', 'cpvu',...   
857  ['investment', 'hold', 'ltd', 'corporate', 'su...   

                            report_lemmatized_filtered  \
321  ['beyond', 'responsibility', 'building', 'comm...   
498  ['fqpsujoh', 'bobhfnfou', 'poz', 'shboj', 'ohb...   
500  ['csr', 'report', 'fqpsujoh', 'fqpsujoh', 'bob...   
502  ['csr', 'report', 'index', 'csr', 'report', 'm...   
505  ['sustainability', 'report', 'uif', 'vtubjobcj...   
857  ['investment', 'hold', 'ltd', 'corporate', 'su...   

                           report_lemmatized_filtered2  nmf_topics  
321  beyond responsibility building community corpo...          26  
498  fqpsujoh bobhfnfou poz shboj ohbhfnfou boe psq...          26  
500  csr report fqpsujoh fqpsujoh bobhfnfou poz shb...          26  
502  csr report index csr report management message...          26  
505  sustainability report uif vtubjobcjmjuz fqpsu ...          26  
857  investment hold ltd corporate sustainability r...          26

In [9]:
# locate the main topic of a specific document (by df index)
pd.DataFrame(nmf_topic_values).loc[513].idxmax()

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


27

In [10]:
test = df['nmf_topics'].value_counts()
test = pd.DataFrame(test).reset_index().rename(columns={'index':'Topic','nmf_topics':'doc_freq'})
test['Topic'] = test['Topic']+1

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
test.head(5)

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic  doc_freq
0      1       106
1      3        91
2     11        70
3     28        63
4     15        63

In [27]:
fig=px.bar(test, x='Topic', y='doc_freq', template='plotly_white', labels={'doc_freq':'Number of documents'},
       color_discrete_sequence=['#533A71'])
fig.update_xaxes(nticks=30)
fig.update_yaxes(categoryorder="total descending")
fig.show()

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [28]:
test.head(5)

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Topic  doc_freq
0      1       106
1      3        91
2     11        70
3     28        63
4     15        63

In [ ]:
stop

----------

# Metrics

In [13]:
# https://towardsdatascience.com/topic-modeling-articles-with-nmf-8c6b2a227a45

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [14]:
from gensim.models import Nmf

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [15]:
texts = [d.split() for d in df.report_lemmatized_filtered2]

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [16]:
# Use Gensim's NMF to get the best num of topics via coherence score

# Create a dictionary; in gensim, a dictionary is a mapping between words and their integer id
dictionary = Dictionary(texts)

# Filter out extremes to limit the number of features
dictionary.filter_extremes(
    no_below=5,
    no_above=0.95,
    keep_n=5000
)

# Create the bag-of-words format (list of (token_id, token_count))
corpus = [dictionary.doc2bow(text) for text in texts]

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [17]:
from tqdm.notebook import tqdm

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [18]:
topic_nums = range(5,51)

coherence_scores = []
coherence_scores2 = []

# Run the nmf model and calculate the coherence score for each number of topics
for num in tqdm(topic_nums):
    #nmf = Nmf(corpus=corpus,num_topics=num,id2word=dictionary,chunksize=2000,passes=5,kappa=1, # kappa=.1
    #          minimum_probability=0.01,w_max_iter=200,w_stop_condition=0.0001,h_max_iter=100,h_stop_condition=0.001,
    #          eval_every=10,normalize=True,random_state=0) #w_max_iter=300
    
    # try this version instead to see if it's less horribly long
    nmf = Nmf(corpus=corpus, num_topics=num, id2word=dictionary, random_state=0) # normalize=True
    
    # Run the coherence model to get the score
    cm = CoherenceModel(model=nmf, texts=texts, dictionary=dictionary, coherence='u_mass') # c_v
    coherence_scores.append(cm.get_coherence())
    coherence_scores2.append(round(cm.get_coherence(), 5))

/Applications/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



  0%|          | 0/46 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
zipped = list(zip(topic_nums, coherence_scores2))

new_df = pd.DataFrame(zipped, columns=['T','coherence'])

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X1 = new_df[['coherence']]
new_df['Coherence normalized'] = min_max_scaler.fit_transform(X1)

In [ ]:
px.line(new_df, x='T', y='Coherence normalized', template="plotly_white",
       color_discrete_sequence=['#533A71'], labels={'variable':'metric','T':'Number of topics'}, width=650, height=400)

# 10 or 20 topics seems best, seed=0

---------------